In [1]:
# !/usr/bin/env python
 
import pandas as pd
 
# 粗略查看数据信息
u_data = pd.read_csv('D:/开课吧课程资料/核心能力提升班商务智能方向004期/L5/L5-code/L5/deepfm/ml-100k/u.data',header=None,sep="\t")
u_genre = pd.read_csv('D:/开课吧课程资料/核心能力提升班商务智能方向004期/L5/L5-code/L5/deepfm/ml-100k/u.genre',header=None,sep="|")
u_info = pd.read_csv('D:/开课吧课程资料/核心能力提升班商务智能方向004期/L5/L5-code/L5/deepfm/ml-100k/u.info',header=None,sep=" ")
u_item = pd.read_csv('D:/开课吧课程资料/核心能力提升班商务智能方向004期/L5/L5-code/L5/deepfm/ml-100k/u.item',error_bad_lines=False,header=None,sep="|")
u_occupation = pd.read_csv('D:/开课吧课程资料/核心能力提升班商务智能方向004期/L5/L5-code/L5/deepfm/ml-100k/u.occupation',header=None)
u_user = pd.read_csv('D:/开课吧课程资料/核心能力提升班商务智能方向004期/L5/L5-code/L5/deepfm/ml-100k/u.user',header=None,sep="|")
 


In [2]:
import pandas as pd
u_data.columns = ['user_id','item_id','rating','timestamp']
u_genre.columns = ['genre','score']
u_item.columns = ['movie_id','movie_title','release_date','video_release_date','IMDb_URL','unknown','Action','Adventure',
                  'Animation','Childrenr\'s','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical',
                  'Mystery','Romance','Sci-Fi','Thriller','War','Western']
u_occupation.columns = ['occupation']
u_user.columns = ['user_id','age','gender','occupation','zip_code']

In [3]:
u_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1682 non-null   int64  
 1   movie_title         1682 non-null   object 
 2   release_date        1681 non-null   object 
 3   video_release_date  0 non-null      float64
 4   IMDb_URL            1679 non-null   object 
 5   unknown             1682 non-null   int64  
 6   Action              1682 non-null   int64  
 7   Adventure           1682 non-null   int64  
 8   Animation           1682 non-null   int64  
 9   Childrenr's         1682 non-null   int64  
 10  Comedy              1682 non-null   int64  
 11  Crime               1682 non-null   int64  
 12  Documentary         1682 non-null   int64  
 13  Drama               1682 non-null   int64  
 14  Fantasy             1682 non-null   int64  
 15  Film-Noir           1682 non-null   int64  
 16  Horror

In [4]:
u_item.drop(['video_release_date'], inplace=True, axis = 1)

In [5]:
 # 去掉occupation为none的记录
nones = u_user[u_user['occupation'] == 'none']
u_user = u_user.drop(nones.index)

In [6]:
# gender中的m、f映射成0、 1
u_user['gender'] = u_user['gender'].map({'M':1, 'F':0})
print(u_user.head())

   user_id  age  gender  occupation zip_code
0        1   24       1  technician    85711
1        2   53       0       other    94043
2        3   23       1      writer    32067
3        4   24       1  technician    43537
4        5   33       0       other    15213


In [7]:
 # 对age进行分段，映射成7组
def age_map(age):
    if age >= 1 and age <= 7: return 1
    if age >= 8 and age <=16: return 2
    if age >=17 and age <= 29: return 3
    if age >= 30 and age <= 39: return 4
    if age >= 40 and age <= 49: return 5
    if age >= 50 and age <= 59: return 6
    if age >= 60: return 7

In [8]:
u_user['age'] = u_user['age'].apply(lambda age : age_map(age))
print(u_user.head())

   user_id  age  gender  occupation zip_code
0        1    3       1  technician    85711
1        2    6       0       other    94043
2        3    3       1      writer    32067
3        4    3       1  technician    43537
4        5    4       0       other    15213


In [9]:
#  occupation字段数值化
def occupations_map(occupation):
    occupations_dict = {'technician': 1,
     'other': 0,
     'writer': 2,
     'executive': 3,
     'administrator': 4,
     'student': 5,
     'lawyer': 6,
     'educator': 7,
     'scientist': 8,
     'entertainment': 9,
     'programmer': 10,
     'librarian': 11,
     'homemaker': 12,
     'artist': 13,
     'engineer': 14,
     'marketing': 15,
     'none': 16,
     'healthcare': 17,
     'retired': 18,
     'salesman': 19,
     'doctor': 20}
    return occupations_dict[occupation]
u_user['occupation'] = u_user['occupation'].apply(lambda occupation : occupations_map(occupation))
print(u_user.head())

   user_id  age  gender  occupation zip_code
0        1    3       1           1    85711
1        2    6       0           0    94043
2        3    3       1           2    32067
3        4    3       1           1    43537
4        5    4       0           0    15213


In [10]:
u_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 934 entries, 0 to 942
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     934 non-null    int64 
 1   age         934 non-null    int64 
 2   gender      934 non-null    int64 
 3   occupation  934 non-null    int64 
 4   zip_code    934 non-null    object
dtypes: int64(4), object(1)
memory usage: 43.8+ KB


In [11]:
#  zip_code提取前3位
u_user['zip_code'] = u_user['zip_code'].apply(lambda zip_code : str(zip_code)[:3])

In [12]:
print(u_user.head())

   user_id  age  gender  occupation zip_code
0        1    3       1           1      857
1        2    6       0           0      940
2        3    3       1           2      320
3        4    3       1           1      435
4        5    4       0           0      152


In [13]:
print(u_item.head())

   movie_id        movie_title release_date  \
0         1   Toy Story (1995)  01-Jan-1995   
1         2   GoldenEye (1995)  01-Jan-1995   
2         3  Four Rooms (1995)  01-Jan-1995   
3         4  Get Shorty (1995)  01-Jan-1995   
4         5     Copycat (1995)  01-Jan-1995   

                                            IMDb_URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0       0   

   Adventure  Animation  Childrenr's  Comedy  ...  Fantasy  Film-Noir  Horror  \
0          0          1            1       1  ...        0          0       0   
1          1          0            0       0  ...        0          0       0   
2          0          0            

In [15]:
#删除u_item中无用的字段
u_item_filter = u_item.drop(['movie_title','IMDb_URL','unknown'], axis = 1)

In [16]:
u_item_filter.head()

,movie_id,release_date,Action,Adventure,Animation,Childrenr's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,01-Jan-1995,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,01-Jan-1995,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,01-Jan-1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,01-Jan-1995,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,01-Jan-1995,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [14]:
print(u_data.head())

   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596


In [27]:
data_merge_user = pd.merge(u_data,u_user,on='user_id',how='left').sort_values(by='user_id')

In [28]:
data_merge_user

,user_id,item_id,rating,timestamp,age,gender,occupation,zip_code
66567,1,55,5,875072688,3.0,1.0,1.0,857
62820,1,203,4,878542231,3.0,1.0,1.0,857
10207,1,183,5,875072262,3.0,1.0,1.0,857
9971,1,150,5,876892196,3.0,1.0,1.0,857
22496,1,68,4,875072688,3.0,1.0,1.0,857
...,...,...,...,...,...,...,...,...
96823,943,427,4,888639147,3.0,1.0,5.0,778
70902,943,12,5,888639093,3.0,1.0,5.0,778
84518,943,284,2,875502192,3.0,1.0,5.0,778
72321,943,62,3,888640003,3.0,1.0,5.0,778


In [31]:
#为了将item数据与data数据merge，需要将item中的movie_id改为item_id
u_item.rename(columns={'movie_id':'item_id'},inplace=True)

In [32]:
# 将data、user、item数据进行合并
data_merge_item = pd.merge(data_merge_user, u_item, on='item_id').sort_values(by='item_id')

In [34]:
data_merge_item

,user_id,item_id,rating,timestamp,age,gender,occupation,zip_code,movie_title,release_date,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
34547,751,1,3,889132162,3.0,0.0,0.0,900,Toy Story (1995),01-Jan-1995,...,0,0,0,0,0,0,0,0,0,0
34440,532,1,5,893119335,3.0,1.0,5.0,927,Toy Story (1995),01-Jan-1995,...,0,0,0,0,0,0,0,0,0,0
34439,526,1,5,885682562,4.0,1.0,15.0,971,Toy Story (1995),01-Jan-1995,...,0,0,0,0,0,0,0,0,0,0
34438,525,1,4,881085964,3.0,0.0,4.0,197,Toy Story (1995),01-Jan-1995,...,0,0,0,0,0,0,0,0,0,0
34437,523,1,5,883701763,6.0,0.0,4.0,041,Toy Story (1995),01-Jan-1995,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,863,1678,1,889289570,3.0,1.0,5.0,600,Mat' i syn (1997),06-Feb-1998,...,0,0,0,0,0,0,0,0,0,0
99995,863,1679,3,889289491,3.0,1.0,5.0,600,B. Monkey (1998),06-Feb-1998,...,0,0,0,0,0,1,0,1,0,0
99997,863,1680,2,889289570,3.0,1.0,5.0,600,Sliding Doors (1998),01-Jan-1998,...,0,0,0,0,0,1,0,0,0,0
99998,896,1681,3,887160722,3.0,1.0,2.0,915,You So Crazy (1994),01-Jan-1994,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# 处理好的数据保存，留待后续直接使用
data_merge_item.to_csv('./movielens_dataset_merge.csv')

In [50]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
# from deepctr.inputs import SparseFeat,get_feature_names
from deepctr.feature_column import SparseFeat,get_feature_names
#数据加载
data = pd.read_csv("./movielens_dataset_merge.csv")
#删除无用字段
data.drop(['timestamp','movie_title','IMDb_URL','unknown'], inplace=True, axis = 1)



In [52]:
data.drop(['Unnamed: 0'], inplace=True, axis = 1)

In [53]:
data.head()

,user_id,item_id,rating,age,gender,occupation,zip_code,release_date,Action,Adventure,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,751,1,3,3.0,0.0,0.0,900,01-Jan-1995,0,0,...,0,0,0,0,0,0,0,0,0,0
1,532,1,5,3.0,1.0,5.0,927,01-Jan-1995,0,0,...,0,0,0,0,0,0,0,0,0,0
2,526,1,5,4.0,1.0,15.0,971,01-Jan-1995,0,0,...,0,0,0,0,0,0,0,0,0,0
3,525,1,4,3.0,0.0,4.0,197,01-Jan-1995,0,0,...,0,0,0,0,0,0,0,0,0,0
4,523,1,5,6.0,0.0,4.0,041,01-Jan-1995,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
data.rename(columns={'Childrenr\'s':'Children'},inplace=True)

In [68]:
sparse_features = ["user_id", "item_id", "age", "gender", "occupation", "zip_code","release_date","Action","Adventure","Animation",
                   "Children","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance",
                   "Sci-Fi","Thriller","War","Western"]
target = ['rating']

In [69]:
data['zip_code'] = data['zip_code'].apply(str)
data['release_date'] = data['release_date'].apply(str)

In [70]:
from sklearn.preprocessing import LabelEncoder
# 对特征标签进行编码
for feature in sparse_features:
#     print(type(feature))
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

In [71]:
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
print(fixlen_feature_columns)
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

[SparseFeat(name='user_id', vocabulary_size=943, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000001C8622189B0>, embedding_name='user_id', group_name='default_group', trainable=True), SparseFeat(name='item_id', vocabulary_size=1682, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000001C862218978>, embedding_name='item_id', group_name='default_group', trainable=True), SparseFeat(name='age', vocabulary_size=908, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000001C862218AC8>, embedding_name='age', group_name='default_group', trainable=True), SparseFeat(name='gender', vocabulary_size=903, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000001C86

C:\Users\admin\AppData\Roaming\Python\Python36\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\admin\AppData\Roaming\Python\Python36\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


64000/64000 [==============================] - 5s 76us/sample - loss: 1.8822 - mse: 1.8822 - val_loss: 0.9129 - val_mse: 0.9128
test RMSE 0.959218431849597
